In [ ]:
!pip install dominate
!pip install visdom


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=7423169ee75e895f380504df7be219e6521bb44f3006c7a0e71231b7731e49aa
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix
!ls


/content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix
checkpoints	datasets	 imgs	   logs     pix2pix.ipynb	 requirements.txt  test.py
CycleGAN.ipynb	docs		 img.webp  models   processed_image.png  results	   train.py
data		environment.yml  LICENSE   options  README.md		 scripts	   util


In [ ]:
!python train.py --dataroot /content/drive/MyDrive/dataset --name pix2pix_custom --model pix2pix --direction AtoB --n_epochs 50 --n_epochs_decay 0 --batch_size 1 --gpu_ids 0

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/dataset	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              

splitting of images

In [7]:
from PIL import Image
import os # Import the os module

def split_image_into_patches(image_path, patch_size=256):
    img = Image.open(image_path)
    width, height = img.size

    patches = []

    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            box = (j, i, j + patch_size, i + patch_size)
            patch = img.crop(box)
            patches.append((patch, box))

    return patches, img.size

# Step 1: Split image into patches
input_image = '/content/drive/MyDrive/AOF_dataset/joint 2_page-0003.jpg'
patches, original_size = split_image_into_patches(input_image)

# Step 2: Process each patch with Pix2Pix
# Create the 'temporary' directory if it doesn't exist
temp_dir = '/content/drive/MyDrive/temporary'
os.makedirs(temp_dir, exist_ok=True) # This line creates the directory

for idx, (patch, box) in enumerate(patches):
    # Save patch as a temporary image
    patch_path = f'/content/drive/MyDrive/temporary/patch_{idx}.jpg'
    patch.save(patch_path)

test with patches

In [9]:
pip install dominate

In [10]:
#!set -ex
!python /content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix/test.py --dataroot /content/drive/MyDrive/temporary --name pix2pix_custom --checkpoints_dir /content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix/checkpoints --model test --netG unet_256 --direction BtoA --dataset_mode single --norm batch

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix/checkpoints	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/temporary	[default: None]
             dataset_mode: single                        
                direction: BtoA                          	[default: AtoB]
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                      

reassemble the patches

In [11]:
import os
from PIL import Image

def reassemble_image(patches, original_size):
    # Create a new blank image to reassemble patches into
    output_image = Image.new('RGB', original_size)

    # Paste each patch into the correct position in the output image
    for patch, box in patches:
        output_image.paste(patch, box)

    return output_image

# Step 3: Reassemble the output image
output_patches = []
for idx, (patch, box) in enumerate(patches):
    processed_patch_path = os.path.join("/content/drive/MyDrive/pytorch-CycleGAN-and-pix2pix/results/pix2pix_custom/test_latest/images", f'patch_{idx}_fake.png')
    if os.path.exists(processed_patch_path):
        processed_patch = Image.open(processed_patch_path)
        output_patches.append((processed_patch, box))  # Use original box coordinates
    else:
        print(f"Warning: Processed patch {processed_patch_path} not found.")

# Reassemble the output image using original_size from Step 1
if output_patches:
    output_image = reassemble_image(output_patches, original_size)
    final_output_path = '/content/output_image_test.jpg'
    output_image.save(final_output_path)
    print(f"Final output image saved to {final_output_path}")
else:
    print("No processed patches found. Unable to create final output image.")

Final output image saved to /content/output_image_test.jpg


Clean the directory

In [12]:
import shutil
import os
temp_folder = '/content/drive/MyDrive/temporary'
if os.path.exists(temp_folder):
    for filename in os.listdir(temp_folder):
        file_path = os.path.join(temp_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Step 6: Clear results folder
results_folder = '/content/results/pix2pix_custom/test_latest/images/'
if os.path.exists(results_folder):
    for filename in os.listdir(results_folder):
        file_path = os.path.join(results_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

print("Cleanup complete.")

Cleanup complete.


crp the image

In [3]:
import os
import cv2

def batch_crop_images(input_dir, output_dir, target_size):
    """
    Batch crop images from input directory to output directory with specified target size.

    Parameters:
    - input_dir: Directory path containing input images.
    - output_dir: Directory path where cropped images will be saved.
    - target_size: Tuple (width, height) of the output image size.
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get list of all files in the input directory
    files = os.listdir(input_dir)

    for file in files:
        # Construct full path for input and output images
        input_path = os.path.join(input_dir, file)

        # Read the image using OpenCV
        img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)

        # Check if image was successfully read
        if img is None:
            print(f"Warning: Could not read image '{input_path}'. Skipping...")
            continue

        # Get original image dimensions
        height, width = img.shape[:2]

        # Calculate center coordinates for cropping
        left = (width - target_size[0]) // 2
        top = (height - target_size[1]) // 2
        right = left + target_size[0]
        bottom = top + target_size[1]

        # Crop the image
        cropped_img = img[top:bottom, left:right]

        # Generate new filename for cropped image
        filename, file_extension = os.path.splitext(file)
        new_filename = f"{filename}_cropped{file_extension}"
        output_path = os.path.join(output_dir, new_filename)

        # Save the cropped image
        cv2.imwrite(output_path, cropped_img)

        print(f"Cropped image saved: {output_path}")

# Example usage:
input_directory = '/content/drive/MyDrive/AOF_dataset'
output_directory = '/content/drive/MyDrive/AOF_dataset_crp'
target_size = (256, 256)

batch_crop_images(input_directory, output_directory, target_size)


Cropped image saved: /content/drive/MyDrive/AOF_dataset_crp/joint 1_page-0009_cropped.jpg
Cropped image saved: /content/drive/MyDrive/AOF_dataset_crp/joint 3_page-0007_cropped.jpg


KeyboardInterrupt: 

handwritten whole clean

In [ ]:
import cv2
import numpy as np
import os

# Specify the input and output directories
input_dir = "/content/drive/MyDrive/handwritten_crp"
output_dir = "/content/drive/MyDrive/handwritten_crp_clean"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(input_dir, filename)

        # Load the image using OpenCV
        try:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)

            if img is None:
                print(f"Error: Unable to load image {filename}")
                continue  # Skip to the next image if loading fails

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Use a binary threshold to create a binary image
            _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

            # Detect horizontal lines
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
            horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

            # Detect vertical lines
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
            vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

            # Combine the detected lines
            lines = cv2.add(horizontal_lines, vertical_lines)

            # Invert the detected lines to get the mask of lines to be removed
            mask = cv2.bitwise_not(lines)

            # Use the mask to remove the lines from the original image
            cleaned_img = cv2.bitwise_and(img, img, mask=mask)

            # Convert the cleaned image back to white background
            cleaned_img[mask == 0] = [255, 255, 255]

            # Save the cleaned image
            cleaned_img_path = os.path.join(output_dir, f"cleaned_{filename}")
            cv2.imwrite(cleaned_img_path, cleaned_img)

            print(f"Image {filename} processed and saved as {cleaned_img_path}")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

print("Processing completed. Cleaned images are saved in:", output_dir)

Image a01-011u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-011u_cropped.png
Image a01-077u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-077u_cropped.png
Image a01-026u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-026u_cropped.png
Image a01-014u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-014u_cropped.png
Image a01-020u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-020u_cropped.png
Image a01-082u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-082u_cropped.png
Image a01-072u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-072u_cropped.png
Image a01-063u_cropped.png processed and saved as /content/drive/MyDrive/handwritten_crp_clean/cleaned_a01-063u_cropped.png
Image a0

super imposing on og crop

In [ ]:
import cv2
import numpy as np
import os
import random

def add_checkboxes(image, num_checkboxes=1, checkbox_size=50, color=(0, 0, 0), thickness=5):
    """
    Adds checkboxes to the image.
    :param image: Input image
    :param num_checkboxes: Number of checkboxes to add
    :param checkbox_size: Size of the checkbox
    :param color: Color of the checkbox lines
    :param thickness: Thickness of the checkbox lines
    :return: Image with checkboxes
    """
    h, w = image.shape[:2]

    for _ in range(num_checkboxes):
        x = random.randint(0, w - checkbox_size)
        y = random.randint(0, h - checkbox_size)
        top_left = (x, y)
        bottom_right = (x + checkbox_size, y + checkbox_size)
        cv2.rectangle(image, top_left, bottom_right, color, thickness)

    return image

def add_centered_dotted_lines(image, num_lines=2, line_length=200, dot_distance=20, color=(0, 0, 0), thickness=2):
    """
    Adds large dotted lines to the center of the image.
    :param image: Input image
    :param num_lines: Number of dotted lines to add
    :param line_length: Length of each dotted line
    :param dot_distance: Distance between dots
    :param color: Color of the dots
    :param thickness: Thickness of the dots
    :return: Image with dotted lines
    """
    h, w = image.shape[:2]
    center_x, center_y = w // 2, h // 2

    for i in range(num_lines // 2):
        y_offset = i * dot_distance * 2
        for j in range(0, line_length, dot_distance):
            cv2.circle(image, (center_x - line_length // 2 + j, center_y - y_offset), 1, color, thickness)
            cv2.circle(image, (center_x - line_length // 2 + j, center_y + y_offset), 1, color, thickness)

    return image

# Load handwritten images
input_dir = '/content/drive/MyDrive/handwritten_crp_clean'
output_dir = '/content/drive/MyDrive/new_handwritten_super_crp'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)

        # Add checkboxes and centered dotted lines
        image_with_checkboxes = add_checkboxes(image.copy(), num_checkboxes=1, checkbox_size=50, color=(0, 0, 0), thickness=5)
        image_with_dotted_lines = add_centered_dotted_lines(image_with_checkboxes.copy(), num_lines=2, line_length=200, dot_distance=20, color=(0, 0, 0), thickness=2)

        # Save the images
        cv2.imwrite(os.path.join(output_dir, f'modified_{filename}'), image_with_dotted_lines)


aof_crp  clean

In [ ]:
import cv2
import numpy as np
import os

# Specify the input and output directories
input_dir = "/content/drive/MyDrive/AOF_dataset_crp"
output_dir = "/content/drive/MyDrive/AOF_dataset_crp_cln"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(input_dir, filename)

        # Load the image using OpenCV
        try:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)

            if img is None:
                print(f"Error: Unable to load image {filename}")
                continue  # Skip to the next image if loading fails

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Use a binary threshold to create a binary image
            _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

            # Detect horizontal lines
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
            horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

            # Detect vertical lines
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
            vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

            # Combine the detected lines
            lines = cv2.add(horizontal_lines, vertical_lines)

            # Detect and remove dotted lines
            dotted_lines_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
            dotted_lines = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, dotted_lines_kernel, iterations=1)
            dotted_lines = cv2.bitwise_and(dotted_lines, lines)

            # Detect and remove checkboxes
            checkbox_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
            checkboxes = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, checkbox_kernel, iterations=1)
            checkboxes = cv2.bitwise_and(checkboxes, lines)

            # Combine all detected elements to be removed
            elements_to_remove = cv2.add(dotted_lines, checkboxes)
            elements_to_remove = cv2.add(elements_to_remove, lines)

            # Invert the detected elements to get the mask of elements to be removed
            mask = cv2.bitwise_not(elements_to_remove)

            # Use the mask to remove the elements from the original image
            cleaned_img = cv2.bitwise_and(img, img, mask=mask)

            # Convert the cleaned image back to white background
            cleaned_img[mask == 0] = [255, 255, 255]

            # Save the cleaned image
            cleaned_img_path = os.path.join(output_dir, f"cleaned_{filename}")
            cv2.imwrite(cleaned_img_path, cleaned_img)

            print(f"Image {filename} processed and saved as {cleaned_img_path}")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

print("Processing completed. Cleaned images are saved in:", output_dir)


Image joint 1_page-0009_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 1_page-0009_cropped.jpg
Image joint 3_page-0007_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 3_page-0007_cropped.jpg
Image joint 4_page-0001_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 4_page-0001_cropped.jpg
Image joint 3_page-0001_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 3_page-0001_cropped.jpg
Image joint 3_page-0003_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 3_page-0003_cropped.jpg
Image joint 3_page-0004_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 3_page-0004_cropped.jpg
Image joint 2_page-0010_cropped.jpg processed and saved as /content/drive/MyDrive/AOF_dataset_crp_cln/cleaned_joint 2_page-0010_cropped.jpg
Image joint 3_page-0

pair the image

In [ ]:
import os
from PIL import Image

def find_matching_file(filename, search_dir):
    """
    Find a file in the search directory that matches the main part of the filename.
    """
    main_part = filename.replace("modified_", "")
    for search_filename in os.listdir(search_dir):
        if search_filename.endswith(".jpg") or search_filename.endswith(".png"):
            if main_part in search_filename:
                return os.path.join(search_dir, search_filename)
    return None

def load_image_pairs(input_dir, output_dir, save_dir=None):
    input_images = []
    output_images = []

    # Ensure save directory exists if provided
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Iterate through files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_path = os.path.join(input_dir, filename)

            # Find the corresponding output file
            output_path = find_matching_file(filename, output_dir)

            # Check if the output file exists before attempting to open it
            if output_path:
                input_image = Image.open(input_path).convert("RGB")
                output_image = Image.open(output_path).convert("RGB")

                input_images.append(input_image)
                output_images.append(output_image)

                # Save the paired image if save_dir is provided
                if save_dir:
                    paired_image = Image.new('RGB', (input_image.width + output_image.width, max(input_image.height, output_image.height)))
                    paired_image.paste(input_image, (0, 0))
                    paired_image.paste(output_image, (input_image.width, 0))

                    save_path = os.path.join(save_dir, f"paired_{os.path.basename(output_path)}")
                    paired_image.save(save_path)
                    print(f"Paired image saved at {save_path}")
            else:
                print(f"Warning: Output file not found for {filename}. Skipping.")

    return input_images, output_images

# Example usage:
input_dir = '/content/drive/MyDrive/handwritten_crp_super'
output_dir = '/content/drive/MyDrive/handwritten_crp'  # Make sure this is the correct directory
save_dir = '/content/drive/MyDrive/train_paired_256_new'

input_images, output_images = load_image_pairs(input_dir, output_dir, save_dir)


Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-011u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-077u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-026u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-014u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-020u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-082u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-072u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-063u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-068u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_256_new/paired_a01-007u_cropped.png
Paired image saved at /content/drive/MyDrive/train_paired_25

final super impose code

In [ ]:
import cv2
import numpy as np

def add_checkboxes(image, positions, checkbox_size=20, color=(0, 0, 0), thickness=1):
    """
    Adds checkboxes to the image at specified positions.
    :param image: Input image
    :param positions: List of (x, y) positions for the checkboxes
    :param checkbox_size: Size of the checkbox
    :param color: Color of the checkbox lines
    :param thickness: Thickness of the checkbox lines
    :return: Image with checkboxes
    """
    for pos in positions:
        top_left = pos
        bottom_right = (pos[0] + checkbox_size, pos[1] + checkbox_size)
        cv2.rectangle(image, top_left, bottom_right, color, thickness)

    return image

def add_dotted_lines(image, line_positions, dot_distance=10, color=(0, 0, 0), thickness=1):
    """
    Adds dotted lines to the image at specified positions.
    :param image: Input image
    :param line_positions: List of (x1, y1, x2, y2) positions for the lines
    :param dot_distance: Distance between dots
    :param color: Color of the dots
    :param thickness: Thickness of the dots
    :return: Image with dotted lines
    """
    for (x1, y1, x2, y2) in line_positions:
        length = int(np.hypot(x2 - x1, y2 - y1))
        for i in range(0, length, dot_distance):
            x = int(x1 + i * (x2 - x1) / length)
            y = int(y1 + i * (y2 - y1) / length)
            cv2.circle(image, (x, y), 1, color, thickness)

    return image

# Load the image
img_path = '/content/drive/MyDrive/handwritten_crp/a01-000u_cropped.png'
image = cv2.imread(img_path, cv2.IMREAD_COLOR)

# Create a copy of the original image
overlay_image = image.copy()

# Define checkbox positions (top-left corner)
checkbox_positions = [(150, 200), (280, 200), (410, 200), (540, 200)]

# Define dotted line positions (start and end points)
dotted_line_positions = [(50, 100, 700, 100), (50, 150, 700, 150)]

# Add checkboxes and dotted lines to the overlay image
image_with_checkboxes = add_checkboxes(overlay_image, checkbox_positions, checkbox_size=20, color=(0, 0, 0), thickness=1)
image_with_dotted_lines = add_dotted_lines(image_with_checkboxes, dotted_line_positions, dot_distance=10, color=(0, 0, 0), thickness=1)

# Save the modified image
modified_img_path = 'mmodified_image1.png'
cv2.imwrite(modified_img_path, image_with_dotted_lines)

modified_img_path


'mmodified_image1.png'

folder super impose

In [ ]:
import cv2
import numpy as np
import os

def add_checkboxes(image, positions, checkbox_size=20, color=(0, 0, 0), thickness=1):
    """
    Adds checkboxes to the image at specified positions.
    :param image: Input image
    :param positions: List of (x, y) positions for the checkboxes
    :param checkbox_size: Size of the checkbox
    :param color: Color of the checkbox lines
    :param thickness: Thickness of the checkbox lines
    :return: Image with checkboxes
    """
    for pos in positions:
        top_left = pos
        bottom_right = (pos[0] + checkbox_size, pos[1] + checkbox_size)
        cv2.rectangle(image, top_left, bottom_right, color, thickness)

    return image

def add_dotted_lines(image, line_positions, dot_distance=10, color=(0, 0, 0), thickness=1):
    """
    Adds dotted lines to the image at specified positions.
    :param image: Input image
    :param line_positions: List of (x1, y1, x2, y2) positions for the lines
    :param dot_distance: Distance between dots
    :param color: Color of the dots
    :param thickness: Thickness of the dots
    :return: Image with dotted lines
    """
    for (x1, y1, x2, y2) in line_positions:
        length = int(np.hypot(x2 - x1, y2 - y1))
        for i in range(0, length, dot_distance):
            x = int(x1 + i * (x2 - x1) / length)
            y = int(y1 + i * (y2 - y1) / length)
            cv2.circle(image, (x, y), 1, color, thickness)

    return image

# Define input and output directories
input_dir = '/content/drive/MyDrive/handwritten_crp'
output_dir = '/content/drive/MyDrive/handwritten_crp_super'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define checkbox positions (top-left corner)
checkbox_positions = [(150, 200), (280, 200), (410, 200), (540, 200)]

# Define dotted line positions (start and end points)
dotted_line_positions = [(50, 100, 700, 100), (50, 150, 700, 150)]

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(input_dir, filename)
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # Create a copy of the original image
        overlay_image = image.copy()

        # Add checkboxes and dotted lines to the overlay image
        image_with_checkboxes = add_checkboxes(overlay_image, checkbox_positions, checkbox_size=20, color=(0, 0, 0), thickness=1)
        image_with_dotted_lines = add_dotted_lines(image_with_checkboxes, dotted_line_positions, dot_distance=10, color=(0, 0, 0), thickness=1)

        # Save the modified image
        modified_img_path = os.path.join(output_dir, f'modified_{filename}')
        cv2.imwrite(modified_img_path, image_with_dotted_lines)

print("Processing completed. Modified images are saved in:", output_dir)


Processing completed. Modified images are saved in: /content/drive/MyDrive/handwritten_crp_super


ocr clean

In [ ]:
import os
import cv2
import numpy as np
import easyocr

def detect_and_keep_text(input_image_path, output_image_path):
    if not os.path.exists(input_image_path):
        raise FileNotFoundError(f"Input image file not found at {input_image_path}")

    # Read the image in grayscale
    image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

    # Initialize EasyOCR reader with GPU enabled
    reader = easyocr.Reader(['en'], gpu=True)

    # Detect text using EasyOCR
    results = reader.readtext(image)

    # Create a blank white image
    result_image = np.ones_like(image) * 255

    # Copy the text regions from the original image to the blank image
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Extract text region from the original image
        text_region = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

        # Place the text region on the result image at the same location
        result_image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = text_region

    # Save the result image
    cv2.imwrite(output_image_path, result_image)

def main(input_image_path, output_image_path):
    detect_and_keep_text(input_image_path, output_image_path)
    print("Image processed successfully.")

# Example usage with provided paths
input_image_path = r"/content/mmodified_image1.png"
output_image_path = r"ocr_cln.png"

main(input_image_path, output_image_path)




Image processed successfully.


our clean

In [ ]:
import cv2
import numpy as np
import os

def clean_image(input_image_path, output_image_path):
    """
    Clean an image by removing horizontal and vertical lines and save it.

    Parameters:
    - input_image_path: Path to the input image.
    - output_image_path: Path where the cleaned image will be saved.
    """
    # Load the image using OpenCV
    img = cv2.imread(input_image_path, cv2.IMREAD_COLOR)

    if img is None:
        raise FileNotFoundError(f"Error: Unable to load image '{input_image_path}'")

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use a binary threshold to create a binary image
    _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Detect horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

    # Detect vertical lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

    # Combine the detected lines
    lines = cv2.add(horizontal_lines, vertical_lines)

    # Invert the detected lines to get the mask of lines to be removed
    mask = cv2.bitwise_not(lines)

    # Use the mask to remove the lines from the original image
    cleaned_img = cv2.bitwise_and(img, img, mask=mask)

    # Convert the cleaned image back to white background
    cleaned_img[mask == 0] = [255, 255, 255]

    # Save the cleaned image
    cv2.imwrite(output_image_path, cleaned_img)

    print(f"Image processed and saved as {output_image_path}")

# Example usage
input_image_path = "/content/ocr_cln.png"
output_image_path = "complete_cln.png"

clean_image(input_image_path, output_image_path)
print("Processing completed. Cleaned image is saved at:", output_image_path)

Image processed and saved as complete_cln.png
Processing completed. Cleaned image is saved at: complete_cln.png
